In [2]:
from pathlib import Path
import json
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Paths
root = Path("..")
data_dir = root / "data"

# Load features and artifacts
feat = pd.read_parquet(data_dir / "features.parquet")
feat["ts_signal"] = pd.to_datetime(feat["ts_signal"])  # ensure
with (data_dir / "feature_cols.json").open("r") as f:
    feature_cols = json.load(f)
reg = joblib.load(data_dir / "model_reg.pkl")

# Last-day test set
last_day = feat["ts_signal"].dt.normalize().sort_values().unique()[-1]
test = feat.loc[feat["ts_signal"].dt.normalize() == last_day].copy()
for col in feature_cols:
    if col not in test.columns:
        test[col] = 0.0

X_test = test[feature_cols]
y_true = test["alpha_decay"].astype(float)
y_pred = reg.predict(X_test)

# Scatter plot
plt.figure(figsize=(5,5))
plt.scatter(y_true, y_pred, s=10, alpha=0.6)
mn = float(np.nanmin([y_true.min(), y_pred.min()]))
mx = float(np.nanmax([y_true.max(), y_pred.max()]))
plt.plot([mn, mx], [mn, mx], 'r--', linewidth=1)
plt.xlabel('True alpha_decay (bps)')
plt.ylabel('Predicted (bps)')
plt.title('Regression: y_true vs y_pred (Test)')
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '../data/features.parquet'

In [ ]:
from pathlib import Path
import json

# Read first 3 explanation cards
explanations_path = Path("..") / "data" / "explanations.jsonl"

cards = []
with explanations_path.open("r") as f:
    for i, line in enumerate(f):
        if i >= 3:
            break
        try:
            cards.append(json.loads(line))
        except json.JSONDecodeError:
            continue

print(json.dumps(cards, indent=2))
